In [1]:
import h2o4gpu
import h2o4gpu.util.import_data as io
import h2o4gpu.util.metrics as metrics
from tabulate import tabulate
import pandas as pd

In [2]:
"""
Import Data for H2O GPU Edition

This function will read in data and prepare it for H2O4GPU's GLM solver

Parameters
----------
data_path : str
             A path to a dataset (The dataset needs to be all numeric)
use_pandas : bool
              Indicate if Pandas should be used to parse
intercept : bool
              Indicate if intercept term is needed
valid_fraction : float
                  Percentage of dataset reserved for a validation set
classification : bool
                  Classification problem?
Returns
-------
If valid_fraction > 0 it will return the following:
    train_x: numpy array of train input variables
    train_y: numpy array of y variable
    valid_x: numpy array of valid input variables
    valid_y: numpy array of valid y variable
    family : string that would either be "logistic" if classification is set to True, otherwise "elasticnet"
If valid_fraction == 0 it will return the following:
    train_x: numpy array of train input variables
    train_y: numpy array of y variable
    family : string that would either be "logistic" if classification is set to True, otherwise "elasticnet"
"""

train_x,train_y,valid_x,valid_y,family=io.import_data(data_path="https://s3.amazonaws.com/h2o-public-test-data/h2o4gpu/open_data/creditcard.csv", 
                                                        use_pandas=True, 
                                                        intercept=True,
                                                        valid_fraction=0.2,
                                                        classification=True)

Reading Data with Pandas
(23999, 25)
Original m=23999 n=24
Size of Train rows=19200 & valid rows=4799
Size of Train cols=24 valid cols=24
Size of Train cols=25 & valid cols=25 after adding intercept column


In [3]:
"""
Set up instance of H2O4GPU's GLM solver with default parameters

Need to pass in `family` to indicate problem type to solve
"""
print("Setting up solver")
model = h2o4gpu.GLM(family=family)

Setting up solver


In [4]:
"""
Fit GLM Solver
"""
print("Solving")
%time model.fit(train_x, train_y)
print("Done Solving")
model.summary()

Solving
CPU times: user 18.4 s, sys: 3.06 s, total: 21.4 s
Wall time: 11.6 s
Done Solving
Logloss per alpha value (-1.00 = missing)

|   Alphas |   Train |   CV |   Valid |
|---------:|--------:|-----:|--------:|
|     0.00 |    0.48 | 0.48 |   -1.00 |
|     0.25 |    0.48 | 0.48 |   -1.00 |
|     0.50 |    0.48 | 0.48 |   -1.00 |
|     0.75 |    0.48 | 0.48 |   -1.00 |
|     1.00 |    0.48 | 0.48 |   -1.00 |


In [5]:
"""
Make predictions on validation set
"""
print("Predictions per alpha")
preds = model.predict(valid_x, valid_y)
print(preds)

Predictions per alpha
[[ 0.76388556  0.14441717  0.14333536 ...,  0.26896134  0.24531743
   0.17781904]
 [ 0.76395208  0.14515376  0.145907   ...,  0.26310593  0.24350125
   0.17840983]
 [ 0.76357019  0.14502008  0.14914224 ...,  0.25567541  0.24099426
   0.18065022]
 [ 0.76870704  0.14425999  0.1522073  ...,  0.25546509  0.24379064
   0.1854841 ]
 [ 0.76885712  0.14338331  0.14736274 ...,  0.25683913  0.24017     0.17752925]]


In [6]:
"""
Get logloss or rmse for validation set per alpha
"""
for i in range(model.n_alphas):
    if family == "logistic":
        print("Logloss for alpha = ",model.alphas_best[i])
        print(metrics.log_loss(valid_y, preds[i]))
    else:
        print("RMSE for alpha = ",model.alphas_best[i])
        print(metrics.rmse(valid_y,preds[i]))

Logloss for alpha =  [ 0.]
0.438981
Logloss for alpha =  [ 0.25]
0.439003
Logloss for alpha =  [ 0.5]
0.439169
Logloss for alpha =  [ 0.75]
0.439402
Logloss for alpha =  [ 1.]
0.439157
